In [1]:
# 라이브러리 정의 파트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from mpl_toolkits import basemap
import os
import glob
import netCDF4

In [ ]:
#### mkprism data check ####

# setting path
path_mkprism = '/home/papalio/test_research/RMSE_TEST_2/DATA/MKPRISE_TA'

# mkprism data sample check



# mkprism data value distribution

In [ ]:
#### obs data check ####

# setting path
path_obs = '/home/papalio/test_research/RMSE_TEST_2/DATA/OBS_TA'


# obs data sample check


# obs data value distribution